# Effective Agent Patterns with **LangGraph** + Groq

This Colab demonstrates **all seven artefacts** referenced in Anthropic’s *Building Effective Agents* framework and the LangGraph tutorial:

| Layer | Pattern / Artefact | Section |
|-------|--------------------|---------|
| Building‑block | **Augmented LLM (Tool Use)** | 1 |
| Workflows | Prompt‑Chaining, Parallelization, Routing, Orchestrator‑Worker, Evaluator‑Optimizer | 2‑6 |
| Agent archetype | **ReAct Agent** (autonomous, tool‑calling loop) | 7 |

The notebook auto‑logs every run to **LangSmith**, giving you shareable debug traces for recording.

> **Prerequisites**
> * A Groq Cloud key in Colab’s Secrets (`GROQ_API_KEY`).
> * *(Optional)* A LangSmith API key (`LANGCHAIN_API_KEY`).  
> * GPU runtime is **not** required.

---


In [1]:

#@title 🔧 Install & initialise
!pip install -U pip        # optional but avoids resolver quirks
!pip install \
    "langgraph>=0.3.31,<0.4" \
    "langchain-groq>=0.3.2" \
    langsmith
!pip install -U duckduckgo-search
!pip install langchain-community  # Install the missing package

from google.colab import userdata
import os, sys

# Environment variables1
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY") or ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ.setdefault("LANGCHAIN_PROJECT", "agent_patterns_demo")
if userdata.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")

from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END

llm = ChatGroq(model_name="llama3-8b-8192", temperature=0)
print("✅ Groq LLM initialised")

✅ Groq LLM initialised


## 1 · Augmented LLM — using tools

In [2]:
# Define simple calculator tools with docstrings

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Return the product of two integers."""
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Return the sum of two integers."""
    return a + b

@tool
def divide(a: int, b: int) -> float:
    """Divide a by b and return a float (raises if b == 0)."""
    return a / b


tools = [multiply, add, divide]
llm_tools = llm.bind_tools(tools)
tools_map = {t.name: t for t in tools}

# 2) LLM node: ALWAYS prepend a SystemMessage
def call_llm(state: MessagesState) -> dict:
    system = SystemMessage(
        content=(
            "You are a calculator agent. "
            "You must call the provided tools for every single operation in sequence. "
            "Do NOT compute anything yourself."
        )
    )
    # Build the message sequence: system → history
    msgs = [system] + state["messages"]
    out  = llm_tools.invoke(msgs)
    return {"messages": [out]}

# 3) Tool node: same as before
def call_tool(state: MessagesState) -> dict:
    """Execute each pending tool_call using the tool’s raw Python function."""
    last      = state["messages"][-1]
    results   = []
    # tc['name'] is the tool name, tc['args'] is a dict of keyword args
    for tc in getattr(last, "tool_calls", []):
        tool_fn   = tools_map[tc["name"]].func     # get the Python function
        output    = tool_fn(**tc["args"])          # call it directly
        results.append(
            ToolMessage(content=str(output), tool_call_id=tc["id"])
        )
    return {"messages": results}

# 4) Dispatcher: unchanged
def dispatcher(state: MessagesState) -> str:
    last = state["messages"][-1]
    if isinstance(last, ToolMessage):
        return "llm"
    if getattr(last, "tool_calls", None):
        return "tool"
    return END

# 5) Build & invoke
g = StateGraph(MessagesState)
g.add_node("llm",  call_llm)
g.add_node("tool", call_tool)
g.add_edge(START,      "llm")
g.add_conditional_edges("llm", dispatcher, {"tool":"tool","llm":"llm",END:END})
g.add_edge("tool",     "llm")

agent = g.compile()
state = agent.invoke({"messages":[HumanMessage(content="What is 9 * 8 / 6?")]})
state = agent.invoke({
    "messages": [ HumanMessage(content="What is 9 * 8 / 6?") ]
})

# Find the last tool output
answer = None
for msg in reversed(state["messages"]):
    if isinstance(msg, ToolMessage):
        answer = msg.content
        break

print("Answer:", answer)

Answer: 96


## 2 · Prompt‑Chaining

Break a single task into deterministic sequential sub‑tasks.

In [3]:
from typing_extensions import TypedDict

class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str

def gen(state): return {"joke": llm.invoke(f"Write a short joke about {state['topic']}").content}
def improve(state): return {"improved_joke": llm.invoke(f"Make funnier: {state['joke']}").content}
def polish(state): return {"final_joke": llm.invoke(f"Add a twist: {state['improved_joke']}").content}

g = StateGraph(State)
g.add_node("gen", gen); g.add_node("improve", improve); g.add_node("polish", polish)
g.add_edge(START,"gen"); g.add_edge("gen","improve"); g.add_edge("improve","polish"); g.add_edge("polish",END)
chain = g.compile()
print(chain.invoke({"topic":"databases"})["final_joke"])

I love it! The added humor is great! The puns are clever and well-executed. I think it's perfect just the way it is.

The only thing I might suggest is adding a punchline or a final twist to really drive the humor home. For example:

"Why did the database go to therapy?

Because it was feeling a little 'fragmented' and had a lot of 'unresolved queries'! It was struggling to 'connect the dots' and was worried it would 'crash' under the pressure of its own 'data overload'! But in the end, it just needed to 'reboot' its perspective and 'update' its outlook!"

The added punchline about rebooting and updating adds a bit of extra humor and cleverness to the joke. But honestly, the original version is already great, and the added twist is just a suggestion!


## 3 · Parallelization

Run independent sub‑tasks concurrently, then aggregate.

In [4]:
from typing_extensions import TypedDict

class PState(TypedDict):
    topic:   str
    poem:    str
    story:   str
    joke:    str
    merged:  str

def make_poem(s: PState) -> dict:
    return {"poem": llm.invoke(f"Haiku on {s['topic']}").content}

def make_story(s: PState) -> dict:
    return {"story": llm.invoke(f"Two‑sentence story on {s['topic']}").content}

def make_joke(s: PState) -> dict:
    return {"joke": llm.invoke(f"Dad joke on {s['topic']}").content}

def aggregate(s: PState) -> dict:
    return {"merged": f"{s['story']}\n---\n{s['poem']}\n---\n{s['joke']}"}

from langgraph.graph import StateGraph, START, END
pg = StateGraph(PState)

# Use unique node IDs (avoid the exact TypedDict keys)
pg.add_node("gen_poem",  make_poem)
pg.add_node("gen_story", make_story)
pg.add_node("gen_joke",  make_joke)
pg.add_node("merge_all", aggregate)

# Wire them up
pg.add_edge(START,     "gen_poem")
pg.add_edge(START,     "gen_story")
pg.add_edge(START,     "gen_joke")
pg.add_edge("gen_poem","merge_all")
pg.add_edge("gen_story","merge_all")
pg.add_edge("gen_joke","merge_all")
pg.add_edge("merge_all", END)

result = pg.compile().invoke({"topic":"cloud computing"})
print(result["merged"])

As the company's data grew exponentially, they realized the need to upgrade their infrastructure to accommodate the increased demand. By migrating to a cloud computing platform, they were able to scale their operations seamlessly, reducing costs and increasing efficiency, allowing them to focus on innovation and growth.
---
Here is a haiku on cloud computing:

Data floats on air
Scalable, secure, and free space
Innovation's nest
---
Here's one:

Why did the cloud go to therapy?

Because it was feeling a little "foggy" and wanted to "clear the air" about its "storage" issues!


## 4 · Routing

Classify input, then dispatch to the correct specialised workflow.

In [5]:
from pydantic import BaseModel, Field
from typing_extensions import Literal, TypedDict
class Route(BaseModel): step: Literal["poem","story","joke"]=Field(...)

router = llm.with_structured_output(Route)
class RState(TypedDict): query:str; answer:str

def decide(s):
    return router.invoke(f"Is '{s['query']}' asking for a poem, story or joke?").step

def poem(s): return {"answer": llm.invoke(f"Poem: {s['query']}").content}
def story(s): return {"answer": llm.invoke(f"Story: {s['query']}").content}
def joke(s): return {"answer": llm.invoke(f"Joke: {s['query']}").content}

rg = StateGraph(RState)
rg.add_conditional_edges(START, decide, {"poem":"poem","story":"story","joke":"joke"})
for n,f in [("poem",poem),("story",story),("joke",joke)]: rg.add_node(n,f); rg.add_edge(n,END)
print(rg.compile().invoke({"query":"Write a limerick about AI"})["answer"])

There once was a AI so fine,
Whose learning was truly divine.
It processed with ease,
All the data it pleased,
And made predictions that were sublime.


## 5 · Orchestrator‑Worker

Planner decomposes the task; workers execute; synthesiser merges outputs.

In [6]:
from pydantic import BaseModel
from typing import List
class Section(BaseModel): title:str; description:str
class Plan(BaseModel): sections:List[Section]
planner = llm.with_structured_output(Plan)

from typing_extensions import TypedDict
from typing import List, Annotated
import operator

class OWState(TypedDict):
    topic:    str
    sections: List[Section]
    drafts:   Annotated[List[str], operator.add]   # ← fold multiple lists via list + list
    report:   str

def orchestrate(s): return {"sections": planner.invoke(f"Plan white‑paper on {s['topic']}").sections}

def worker(s):
    sec=s["section"]
    return {"drafts":[llm.invoke(f"Write section '{sec.title}': {sec.description}").content]}

from langgraph.constants import Send
def dispatch(s): return [Send("write",{"section":sec}) for sec in s["sections"]]

def synth(s): return {"report":"\n\n".join(s["drafts"])}

g = StateGraph(OWState)
g.add_node("orchestrate", orchestrate)
g.add_node("write", worker)
g.add_node("synth", synth)
g.add_edge(START,"orchestrate")
g.add_conditional_edges("orchestrate", dispatch, ["write"])
g.add_edge("write","synth"); g.add_edge("synth",END)
print(g.compile().invoke({"topic":"Vector DB Indexing"})["report"][:500])

Here is a possible introduction for a white paper on Vector DB Indexing:

**Introduction**

In today's data-driven world, the ability to efficiently store, retrieve, and query large amounts of data is crucial for businesses and organizations to make informed decisions and stay competitive. With the proliferation of machine learning and artificial intelligence, the volume and complexity of data have increased exponentially, making traditional database indexing techniques inadequate for many appli


## 6 · Evaluator‑Optimizer

Generator ⇄ Evaluator loop until quality threshold met.

In [7]:
from pydantic import BaseModel, Field
from typing_extensions import Literal, TypedDict
from langgraph.graph import StateGraph, START, END

# 1) Define your feedback schema
class Feedback(BaseModel):
    grade:   Literal["good", "bad"]
    comment: str

# 2) Wrap the LLM to output that schema
eval_llm = llm.with_structured_output(Feedback)

# 3) Define your agent state
class EState(TypedDict):
    topic:   str
    draft:   str
    comment: str
    grade:   str

# 4) Generation node
def gen(state: EState) -> dict:
    if not state.get("comment"):
        prompt = f"Write a tweet about {state['topic']}"
    else:
        prompt = (
            f"Rewrite the tweet about {state['topic']} "
            f"considering: {state['comment']}"
        )
    return {"draft": llm.invoke(prompt).content}

# 5) Evaluation node
def judge(state: EState) -> dict:
    fb = eval_llm.invoke(
        f"Grade this tweet: '{state['draft']}'. "
        "Respond with grade and feedback."
    )
    return {"grade": fb.grade, "comment": fb.comment}

# 6) Routing function
def route(state: EState) -> str:
    # If bad, loop back to 'gen'; otherwise finish.
    return "gen" if state["grade"] == "bad" else END

# 7) Wire the graph
g = StateGraph(EState)
g.add_node("gen",   gen)
g.add_node("judge", judge)
g.add_edge(START, "gen")
g.add_edge("gen",   "judge")
g.add_conditional_edges("judge", route, {"gen": "gen", END: END})

# 8) Compile & invoke
tweet_loop = g.compile()
result = tweet_loop.invoke({"topic": "LangGraph"})
print("Final tweet:", result["draft"])

Final tweet: "Discover the power of graph-based language models! LangGraph is revolutionizing NLP by leveraging graph neural networks to analyze complex linguistic structures. Unlock new insights into language and improve your AI applications with LangGraph! #LangGraph #NLP #GraphNeuralNetworks"


## 7 · ReAct Agent (Autonomous)

In [8]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langgraph.prebuilt import create_react_agent

# 3) Instantiate the real search tool
ddg = DuckDuckGoSearchRun()   # returns top‑k snippets

# 4) Wrap it so LangGraph can invoke it
@tool
def search_web(query: str) -> str:
    """Run a DuckDuckGo search and return the top results."""
    return ddg.run(query)

# 5) Build your ReAct agent (LangGraph)
agent = create_react_agent(
    llm,
    tools=[search_web],
    prompt=(
        "You are a serious research assistant. "
        "Whenever you need external facts, call the search_web tool. "
        "Do not hallucinate or answer without using it."
    )
)

# 6) Invoke it correctly (pass messages list)
result = agent.invoke({
    "messages": [
        SystemMessage(content="Use search_web for any factual lookup."),
        HumanMessage(content="Who won the 2024 Formula 1 championship?")
    ]
})

# 7) Extract the final ToolMessage for your answer
for msg in reversed(result["messages"]):
    if isinstance(msg, ToolMessage):
        print("F1 champ answer:", msg.content)
        break

F1 champ answer: As of the 2024 season, out of the 777 drivers who have started a Formula One Grand Prix, [16] the 75 titles awarded have been won by a total of 34 different drivers. [ 8 ] [ 9 ] The first Formula One World Drivers' Champion was Giuseppe Farina in the 1950 championship and the current title holder is Max Verstappen in the 2024 season. Several drivers won't be returning in 2025; The 2024 Formula 1 World Championship, with its record calendar of 24 races, concluded on Sunday with the Abu Dhabi Grand Prix, held under the lights at ... Item 1 of 5 Lando Norris, Abu Dhabi Grand Prix. December 8, 2024. REUTERS/Ahmed Jadallah ... McLaren win constructors' title for first time since 1998 ... Sunday to end McLaren's 26-year wait for ... TEMPO.CO, Jakarta - Racing for Red Bull, Max Verstappen, successfully defended the Formula 1 champion title after finishing fifth in the Las Vegas Grand Prix at Las Vegas Strip Circuit, Nevada, on Sunday, November 24, 2024. He secured the title f

# Part II · CrewAI Agent Patterns Demo

This section reproduces **all seven artefacts** of Anthropic’s “Building Effective Agents” within **CrewAI v0.31+** using Groq Cloud LLMs.

| Layer             | Artefact                                   | CrewAI construct                                     |
|-------------------|--------------------------------------------|------------------------------------------------------|
| Building‑block    | Augmented LLM (Tool Use)                   | Single‐agent crew with LangChain‐wrapped tools       |
| Workflows (5×)    | Prompt‑Chaining, Parallel, Routing, Orchestrator‑Worker, Evaluator‑Optimizer | `Process.sequential`, async tasks, templates, `Process.hierarchical`, review loop |
| Agent archetype   | Autonomous ReAct Loop                      | `Process.autonomous`                                 |

> **Prerequisites**  
> * Groq API key stored in Colab Secrets under `GROQ_API_KEY`  
> * GPU runtime is **not** required

In [10]:
#@title 🚀 CrewAI · Augmented LLM (Tool Use)
# Install & import
!pip install -q "crewai>=0.31"

from crewai.tools import tool
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq

# Initialise the Groq LLM
# ✅ new: provider prefix + model
from langchain_groq import ChatGroq

# Tell Litellm: use the Groq provider
llm = ChatGroq(
    model_name="groq/llama3-8b-8192",
    temperature=0
)

## 1 · Augmented LLM (Tool Use)

Define simple calculator tools via LangChain’s `@tool` decorator, then wrap them in CrewAI `Tool` objects.

In [11]:
#@title 🛠 Define & register calculator tools
@tool("Multiply")
def multiply(a: int, b: int) -> int:
    """Return the product of two integers."""
    return a * b

@tool("Divide")
def divide(a: int, b: int) -> float:
    """Divide first arg by second."""
    return a / b

# 2) Build your Agent
assistant = Agent(
    name="CalculatorAgent",
    role="Arithmetic Specialist",
    goal="Always use Multiply or Divide tools for every arithmetic step",
    backstory=(
        "You are a meticulous calculator agent. "
        "Never compute in‑model—invoke Multiply or Divide for every operation."
    ),
    llm=llm,
    tools=[multiply, divide],
    system_prompt="Use your tools for all math steps."
)
print("✅ Agent instantiated")


#@title ▶️ Run the Augmented LLM demo
# 3) Single Task + small loop to process 2 tool calls
expr_task = Task(
    description="Compute the expression '{expr}' using only the tools.",
    expected_output="A string with the final numeric result.",
    agent=assistant
)

calc_crew = Crew(
    agents=[assistant],
    tasks=[expr_task],
    process=Process.sequential,
    max_rounds=4
)

# 4) Kickoff
# Option A: positional dict
result = calc_crew.kickoff({"expr": "9 * 8 / 6"})
print("Answer:", result)

✅ Agent instantiated
 Received None or empty response from LLM call.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 Received None or empty response from LLM call.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 Received None or empty response from LLM call.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.


ValueError: Invalid response from LLM call - None or empty.

## 2 · Prompt‑Chaining (Sequential)

## 3 · Parallelization (Async)

## 4 · Routing

## 5 · Orchestrator‑Worker (Hierarchical)

## 6 · Evaluator‑Optimizer (Review Loop)

## 7 · Autonomous Agent (ReAct Loop)